In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
import base64
from dash.dependencies import Input, Output, State
from dash import callback_context
import json

from collections import OrderedDict
from bson.json_util import dumps

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#imports AnimalShelter Class from animal_shelter2 Python module
from animal_shelter2 import AnimalShelter



###########################
# Data Manipulation / Model
###########################

#username and password fed to AnimalShelter class Python module
username = "aacuser2"
password = "hello"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#Adds in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    #enodes Grazioso Salvare’s logo with URL href
    html.A([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                    'height' : '20%',
                    'width' : '20%',
                    'float' : 'left',
                    'position' : 'relative',
                    'padding-top' : 0,
                    'padding-right' : 0
            })
    ], href='https://www.snhu.edu/'),
    html.Br(),
    html.Hr(),
    html.Center(html.B(html.H1('Spencer Hayden: CS340 Dashboard'))),
    html.Hr(),
    html.Div(className='row',
             style={'display' : 'flex'},
                 children=[
                    html.Button(id='button-one', n_clicks=0, children='Water Rescue'),
                    html.Button(id='button-two', n_clicks=0, children='Mountain Rescue'),
                    html.Button(id='button-three', n_clicks=0, children='Disaster Rescue'),
                    html.Button(id='button-four', n_clicks=0, children='Reset'),
                
            ]),
  
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {'id': i, 'name': i} for i in df.columns
        ],
        data=df.to_dict('records'),
        #Makes table horizontally scrollable
        style_table={'overflowX': 'auto'},
        #sets cell size
        style_cell={'height': 'auto', 'minWidth': '180px', 'width': '180px',
                    'maxWidth': 'auto', 'whiteSpace': 'normal'},
        style_cell_conditional=([
            {
                'if':{'column_id': i},
                'textAlign': 'left'
            } for i in ['animal_type', 'name', 'breed', 'color', 
                        'outcome_subtype', 'outcome_type', 
                        'sex_upon_outcome']
        ]),
        editable=False,
        filter_action="none",
        sort_action="native",
        sort_mode="multi",
        column_selectable=True,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,

    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
    
])

#############################################
# Interaction Between Components / Controller
#############################################

# Filters interactive data table with MongoDB queries through buttons.
@app.callback(
    Output('datatable-interactivity','data'),
    [Input('button-one', 'n_clicks'),
    Input('button-two', 'n_clicks'),
    Input('button-three', 'n_clicks'),
    Input('button-four', 'n_clicks')]
)
def update_dashboard(btn1, btn2, btn3, btn4):

    changed_id = [p['prop_id'] for p in callback_context.triggered][0]
    if 'button-one' in changed_id:
        df = pd.DataFrame(list(shelter.read({'$or': [{'breed':'Labrador Retriever'},
                                                    {'breed':'Chesapeake Bay Retriever'},
                                                    {'breed':'Newfoundland'}],
                                                    'age_upon_outcome_in_weeks': 
                                                    {'$gte':26,'$lte':156},
                                                    'sex_upon_outcome':'Intact Female'})))
                              
    elif 'button-two' in changed_id:
        df = pd.DataFrame(list(shelter.read({'$or': [{'breed':'German Shepherd'},
                                                    {'breed':'Malamute'},
                                                    {'breed':'Old English Sheepdog'},
                                                    {'breed':'Siberian Husky'},
                                                    {'breed':'Rottweiler'}],
                                                    'age_upon_outcome_in_weeks': 
                                                    {'$gte':26,'$lte':156},
                                                    'sex_upon_outcome':'Intact Male'})))
    elif 'button-three' in changed_id:
        df = pd.DataFrame(list(shelter.read({'$or': [{'breed':'Doberman Pinscher'},
                                                    {'breed':'German Shepherd'},
                                                    {'breed':'Golden Retriever'},
                                                    {'breed':'Bloodhound'},
                                                    {'breed':'Rottweiler'}],
                                                    'age_upon_outcome_in_weeks': 
                                                    {'$gte':20,'$lte':300},
                                                    'sex_upon_outcome':'Intact Male'})))
                                             
    elif 'button-four' in changed_id:
        df = pd.DataFrame.from_records(shelter.read({}))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
      
    return df.to_dict('records')

@app.callback(
    Output('datatable-interactivity', 'style_data_conditional'),
    [Input('datatable-interactivity', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#Updates Pie Chart based on row selection
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-interactivity', "selected_rows")],
    [State('datatable-interactivity', "derived_viewport_data")]
)
def update_graphs(selected_rows, viewData):
    if selected_rows is None:
        selected_rows = []
        
    if viewData is None:
        dff = df
        
    else:
        dff = pd.DataFrame.from_dict(selected_rows)
    
    return [
        dcc.Graph(id='pie-chart',            
            figure = px.pie(
                data_frame=df, 
                values= {'outcome_type': 'Adoption'}, 
                names='breed',
                color='breed',
            ))
    ]
    
        
            
#Updates Geolocation Map based on row selection
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-interactivity', "selected_rows")],
    [State('datatable-interactivity', "derived_viewport_data")]
)
    
def update_map(selected_rows,viewData):
    if selected_rows is None:
        selected_rows = []
        
    if viewData is None:
        dff = df
        
    else:
        dff = pd.DataFrame.from_dict(selected_rows)
        
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[0,13],dff.iloc[0,14]], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
                dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[               
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1(list(shelter.read({'animal_id'}))),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]    
        
    
app

ServerSelectionTimeoutError: localhost:37211: [Errno 111] Connection refused